# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [32]:
!pip install scikeras

In [33]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from scikeras.wrappers import KerasRegressor

In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [34]:
# Import the data
data = pd.read_csv('loan_final.csv')

# Drop rows with no target value
data = data.dropna(subset=['total_pymnt'])

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [35]:
# Print the dimensions of data 
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [36]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [37]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer(missing_values=np.nan, strategy='mean')

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = ss_X.fit_transform(X_train_imputed)

# Transform test data
X_test_scaled = ss_X.transform(X_test_imputed)

# Convert the final results into DataFrames
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=cont_features)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=cont_features)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [38]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat = X_train_cat.fillna('missing')
X_test_cat = X_test_cat.fillna('missing')



# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names_out(input_features=X_train_cat.columns)

# Convert the final results into DataFrames
X_train_categorical = pd.DataFrame(X_train_ohe, columns=cat_columns)
X_test_categorical = pd.DataFrame(X_test_ohe, columns=cat_columns)

Run the below cell to combine the numeric and categorical features. 

In [39]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled_df, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled_df, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [41]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [42]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [43]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model, epochs=150, batch_size=256)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [44]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv=5)

Epoch 1/150


C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7851 - mse: 0.7851 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3556 - mse: 0.3556
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2926 - mse: 0.2926
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2676 - mse: 0.2676
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2519 - mse: 0.2519
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2380 - mse: 0.2380
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2366 - mse: 0.2366
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2303 - mse: 0.2303
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2233 - mse: 0.2233
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2166 - mse: 0.2166
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2154 - mse: 0.2154
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2162 - mse: 0.2162
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5919 - mse: 0.5919 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2707 - mse: 0.2707
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2508 - mse: 0.2508
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2329 - mse: 0.2329
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2323 - mse: 0.2323
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2330 - mse: 0.2330
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2259 - mse: 0.2259
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2178 - mse: 0.2178
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2194 - mse: 0.2194
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2198 - mse: 0.2198
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2151 - mse: 0.2151
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2163 - mse: 0.2163
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

...

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0275 - mse: 1.0275 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3556 - mse: 0.3556
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2608 - mse: 0.2608
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2449 - mse: 0.2449
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2391 - mse: 0.2391
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2230 - mse: 0.2230
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2354 - mse: 0.2354
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2184 - mse: 0.2184
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2150 - mse: 0.2150
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2176 - mse: 0.2176
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2146 - mse: 0.2146
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2175 - mse: 0.2175
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0084 - mse: 1.0084 
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3058 - mse: 0.3058
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2760 - mse: 0.2760
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2549 - mse: 0.2549
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2523 - mse: 0.2523
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2313 - mse: 0.2313
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2331 - mse: 0.2331
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2320 - mse: 0.2320
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2206 - mse: 0.2206
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2190 - mse: 0.2190
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2151 - mse: 0.2151
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2229 - mse: 0.2229
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1068 - mse: 1.1068
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3145 - mse: 0.3145
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2841 - mse: 0.2841
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2527 - mse: 0.2527
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2482 - mse: 0.2482
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2472 - mse: 0.2472
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2369 - mse: 0.2369
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2296 - mse: 0.2296
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2273 - mse: 0.2273
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2247 - mse: 0.2247
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2263 - mse: 0.2263
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2216 - mse: 0.2216
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

- Find the RMSE on train data 

In [21]:
# RMSE on train data (scaled)
rmse_train = np.sqrt(mean_squared_error(y_train_scaled, cv_baseline_preds))
rmse_train

0.4448507566935619

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [22]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
rmse_train_original_scale = np.sqrt(mean_squared_error(y_train, baseline_preds))
rmse_train_original_scale

4051.94069492567

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [45]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_shape=(n_features,)))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return model

In [46]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model, epochs=150, batch_size=256)

In [ ]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled, cv=5)

Epoch 1/150


C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3222 - mse: 0.3222
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1973 - mse: 0.1973
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1941 - mse: 0.1941
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1960 - mse: 0.1960
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1838 - mse: 0.1838
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1928 - mse: 0.1928
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1884 - mse: 0.1884
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1804 - mse: 0.1804
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1912 - mse: 0.1912
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1893 - mse: 0.1893
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1823 - mse: 0.1823
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1773 - mse: 0.1773
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/st

C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.3422 - mse: 0.3422
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2020 - mse: 0.2020
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1928 - mse: 0.1928
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1891 - mse: 0.1891
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1840 - mse: 0.1840
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2005 - mse: 0.2005
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1930 - mse: 0.1930
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1901 - mse: 0.1901
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1812 - mse: 0.1812
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1878 - mse: 0.1878
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1847 - mse: 0.1847
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1953 - mse: 0.1953
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/st

C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3186 - mse: 0.3186
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1989 - mse: 0.1989
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1838 - mse: 0.1838
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1829 - mse: 0.1829
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1931 - mse: 0.1931
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1835 - mse: 0.1835
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1958 - mse: 0.1958
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1888 - mse: 0.1888
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1884 - mse: 0.1884
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1781 - mse: 0.1781
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1794 - mse: 0.1794
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1866 - mse: 0.1866
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/st

C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.3100 - mse: 0.3100
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2060 - mse: 0.2060
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1904 - mse: 0.1904
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1924 - mse: 0.1924
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2073 - mse: 0.2073
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1883 - mse: 0.1883
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1901 - mse: 0.1901
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1953 - mse: 0.1953
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1858 - mse: 0.1858
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1911 - mse: 0.1911
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1943 - mse: 0.1943
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1770 - mse: 0.1770
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/st

In [ ]:
# RMSE on train data (scaled)
rmse_train_data = np.sqrt(mean_squared_error(y_train_scaled, cv_bigger_model_preds))
rmse_train_data

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [ ]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    model = models.Sequential()
    model.add(layers.Dense(10, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                           input_shape=(n_features,)))
    model.add(layers.Dropout(0.2))  # Add dropout layer
    model.add(layers.Dense(5, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dropout(0.2))  # Add dropout layer
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return model

In [ ]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model, epochs=150, batch_size=256)

In [ ]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled, cv=5)

In [ ]:
# RMSE on train data (scaled)
rmse_train = np.sqrt(mean_squared_error(y_train_scaled, cv_dropout_preds))

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [ ]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine

In [ ]:
final_model = create_regularized_model()
final_model.fit(X_train_all, y_train_scaled, epochs=150, batch_size=256, verbose=0)

# Make predictions on the test set
y_test_pred_scaled = final_model.predict(X_test_all)

# Inverse transform the predictions to the original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# Evaluate the model on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 